In [16]:
skip_training = False

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import tools

In [18]:
device = torch.device('cpu')
# device = torch.device('cuda:0')

In [19]:
data_dir = tools.select_data_dir()

The data directory is ../data


In [20]:
# FashionMNIST Dataset
transform = transforms.Compose([
    transforms.ToTensor(),  # Transform to tensor
    transforms.Normalize((0.5,), (0.5,))  # Scale images to [-1, 1]
])

trainset = torchvision.datasets.FashionMNIST(root=data_dir, train=True, download=True, transform=transform)
testset = torchvision.datasets.FashionMNIST(root=data_dir, train=False, download=True, transform=transform)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=5, shuffle=False)

In [21]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6,  out_channels=16, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=256, out_features=120)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.relu4 = nn.ReLU()
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        """
        Args:
          x of shape (batch_size, 1, 28, 28): Input images.
        
        Returns:
          y of shape (batch_size, 10): Outputs of the network.
        """
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        x = self.relu4(x)
        y = self.fc3(x)
        return y

In [22]:
def test_LeNet5_shapes():
    net = LeNet5()

    # Feed a batch of images from the training data to test the network
    with torch.no_grad():
        images, labels = iter(trainloader).next()
        print('Shape of the input tensor:', images.shape)

        y = net(images)
        assert y.shape == torch.Size([trainloader.batch_size, 10]), "Bad shape of y: y.shape={}".format(y.shape)

    print('Success')

test_LeNet5_shapes()

Shape of the input tensor: torch.Size([32, 1, 28, 28])
Success


In [23]:
# This function computes the accuracy on the test dataset
def compute_accuracy(net, testloader):
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [24]:
# Create network
net = LeNet5()
# Training Loop
if not skip_training:
    optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    epochs = 10
    total_loss = 0.0
    
    for epoch in range(epochs):
        print('Epoch: {}'.format(epoch))
        for idx, (train_x, train_label) in enumerate(trainloader):
            optimizer.zero_grad()
            predict_y = net(train_x.float())
            loss = criterion(predict_y, train_label.long())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            if idx % 100 == 0:
                print('idx: {}, loss: {}'.format(idx, loss))

Epoch: 0
idx: 0, loss: 2.304081439971924
idx: 100, loss: 2.309295654296875
idx: 200, loss: 2.2801530361175537
idx: 300, loss: 2.2892472743988037
idx: 400, loss: 2.256218671798706
idx: 500, loss: 2.172274589538574
idx: 600, loss: 2.0260114669799805
idx: 700, loss: 1.4036506414413452
idx: 800, loss: 1.11598539352417
idx: 900, loss: 0.7152453660964966
idx: 1000, loss: 0.9190495014190674
idx: 1100, loss: 0.7992465496063232
idx: 1200, loss: 0.8417239785194397
idx: 1300, loss: 0.9081864356994629
idx: 1400, loss: 0.567398190498352
idx: 1500, loss: 0.9571714997291565
idx: 1600, loss: 0.770348310470581
idx: 1700, loss: 0.6262383460998535
idx: 1800, loss: 0.9207879304885864
Epoch: 1
idx: 0, loss: 0.6547536253929138
idx: 100, loss: 0.6201761364936829
idx: 200, loss: 0.49556052684783936
idx: 300, loss: 0.7188724875450134
idx: 400, loss: 0.7941605448722839
idx: 500, loss: 0.5893073678016663
idx: 600, loss: 0.5605278015136719
idx: 700, loss: 0.955479621887207
idx: 800, loss: 0.5693843960762024
idx: 

In [25]:
# Save the model to disk
if not skip_training:
    tools.save_model(net, '1_lenet5.pth')
else:
    net = LeNet5()
    tools.load_model(net, '1_lenet5.pth', device)

Do you want to save the model (type yes to confirm)? yes
Model saved to 1_lenet5.pth.


In [26]:
# Compute the accuracy on the test set
accuracy = compute_accuracy(net, testloader)
print('Accuracy of the network on the test images: %.3f' % accuracy)
assert accuracy > 0.85, "Poor accuracy {:.3f}".format(accuracy)
print('Success')

Accuracy of the network on the test images: 0.873
Success
